다음 강의는 [DeepLearningAI의 Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/4/create-agents-to-research-and-write-an-article-(code)) 입니다.

# 1. 연구 및 기사 작성을 위한 에이전트 생성

이제 연구 및 기사 작성을 위한 에이전트를 생성해 보겠습니다. 이 에이전트들은 주제에 대한 정보를 수집하고 분석하여 풍부하고 흥미로운 기사를 작성하는 데 도움이 될 것입니다. 에이전트들은 다양한 소스에서 정보를 수집하고 필요한 내용을 요약하여 제공할 수 있습니다. 또한 에이전트들은 특정 주제에 대한 전문 지식을 가지고 있어 정확하고 신뢰할 수 있는 정보를 제공할 수 있습니다.

에이전트들은 자연어 처리 및 기계 학습 기술을 사용하여 효율적으로 정보를 수집하고 분석할 수 있습니다. 또한 에이전트들은 사용자의 요구에 맞게 커스터마이징되어 특정 주제나 관심사에 대한 정보를 제공할 수 있습니다.

이제 연구 및 기사 작성을 위한 에이전트를 생성하여 효율적이고 효과적인 작업을 수행할 수 있도록 해보세요.안녕하세요! 무엇을 도와드릴까요?이 강의에서는 다중 에이전트 시스템의 기본 개념을 소개하고 crewAI 프레임워크에 대한 개요를 얻게 될 것입니다.

- [OPEN API key 생성 바로가기](https://platform.openai.com/api-keys)
- [GROQ API key 생성 바로가기](https://console.groq.com/keys)

다음 명령어를 사용하여 crewai, crewai_tools 및 langchain_community 패키지를 설치할 수 있습니다.

- 가상환경만들기 및 실행하기
```
conda create -n crewai python=3.10
conda activate crewai
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_openai ipykernel langchain python-dotenv
python -m ipykernel install --user --name=crewai
```
- 텍스트문서 하나를 열고 api키를 저장 후 .env파일을 만들어서 디렉토리에 보관하세요

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 크루AI 라이브러리에서 가져오기.

In [3]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
llm2 = ChatGroq(model="llama3-70b-8192",
               api_key = os.getenv("GROQ_API_KEY"))

- 에이전트들을 위한 LLM으로 OpenAI의 `gpt-3.5-turbo`를 사용하게 될 것입니다.

**참고 사항:** crewAI는 에이전트의 LLM으로 다른 인기있는 모델도 사용할 수 있습니다. 몇 가지 예시는 노트북의 [맨 아래](#1)에서 확인할 수 있습니다.

- CrewAI는 Agent, Task 를 각각 만들고 디테일하게 정의하여 연결 후, crew 객체에 모아서 실행하게 됩니다.
    - 각 Agent는 명확하게 자신의 역할대로, 맥락을 이해하며 업무 수행
    - 각 task는 디테일하게 요건을 명확화 하여 정의

![crew2.png](img/crew2.png)

## 에이전트 생성

에이전트는 사용자와 상호작용하고 작업을 수행하는 프로그램입니다. 에이전트를 만들 때는 몇 가지 중요한 요소를 고려해야 합니다. 에이전트의 목적과 기능을 명확히 이해하고, 사용자와의 상호작용 방식을 결정해야 합니다.

에이전트를 만들 때 고려해야 할 몇 가지 중요한 단계은 다음과 같습니다:

1. **목적 설정**: 에이전트가 수행해야 하는 작업과 목표를 명확히 정의합니다.
2. **상호작용 방식 결정**: 에이전트가 사용자와 어떻게 상호작용할지 결정합니다. 이는 대화형 인터페이스, 텍스트 기반 인터페이스, 음성 인식 등 다양한 형태로 이루어질 수 있습니다.
3. **기술 선택**: 에이전트를 구현하기 위해 어떤 기술을 사용할지 결정합니다. 이는 자연어 처리, 기계 학습, 강화 학습 등 다양한 기술을 포함할 수 있습니다.
4. **테스트 및 평가**: 에이전트를 테스트하고 성능을 평가하여 사용자 경험을 향상시키는 데 도움이 됩니다.



### Agent: Planner
참고 : 여러 문자열을 사용하는 이점은 다음과 같습니다.

<br><br>
varname = "텍스트의 1번째 줄"<br>
          "텍스트의 2번째 줄"<br>
<br>
세 개의 따옴표로 된 문서 문자열과 대조적으로
<br>

varname = """텍스트의 1번째 줄<br>
             텍스트의 2번째 줄<br>
          """<br><br>

여러 문자열을 사용하면 공백과 줄 바꿈 문자를 추가하지 않아 LLM에 전달하기에 더 나은 형식으로 만들 수 있습니다.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You are good at Korean language.",
    llm=llm, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

### Agent:Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You are good at Korean language.",
    llm=llm2, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

### Agent:Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible."
              "You are good at Korean language.",
    llm=llm2, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

## 작업 생성

작업을 만들 때는 명확하고 구체적인 목표를 설정하는 것이 중요합니다. 작업을 생성할 때 고려해야 할 몇 가지 중요한 단계가 있습니다.

1. **목표 설정**: 작업의 목표를 명확하게 정의하고 어떤 결과물을 얻고자 하는지 결정합니다.

2. **작업 내용 정의**: 작업의 내용을 상세히 기술하여 무엇을 해야 하는지 명확하게 이해할 수 있도록 합니다.

3. **마감일 설정**: 작업을 완료해야 하는 기한을 설정하여 시간을 효율적으로 관리합니다.

4. **우선순위 설정**: 작업의 중요도에 따라 우선순위를 정하고 중요한 작업을 먼저 처리합니다.

5. **할당 및 추적**: 작업을 담당자에게 할당하고 진행 상황을 추적하여 작업이 원활하게 진행되도록 합니다.

작업을 만들 때 이러한 단계를 따르면 작업을 효율적으로 관리하고 목표를 달성하는 데 도움이 됩니다.Hello! How can I assist you today?**Task 1: Data Entry**

- **Description:** Enter customer information into the database.
- **Expected Output:** Accurate and complete customer profiles in the database.
- **Agent:** Data entry specialist.

**Task 2: Customer Support**

- **Description:** Respond to customer inquiries and provide assistance.
- **Expected Output:** Satisfied customers with their questions answered and issues resolved.
- **Agent:** Customer support representative.

**Task 3: Social Media Management**

- **Description:** Create and schedule posts on social media platforms.
- **Expected Output:** Engaging and consistent social media presence.
- **Agent:** Social media manager.

**Task 4: Sales Calls**

- **Description:** Make outbound sales calls to potential customers.
- **Expected Output:** Increased sales and customer acquisition.
- **Agent:** Sales representative.

### 작업: 계획

1. 문제를 분석하고 이해한다.
2. 해결책을 찾기 위해 가능한 방법을 고려한다.
3. 해결책을 구현하기 위한 단계별 계획을 세운다.
4. 계획을 실행하고 문제를 해결한다.
5. 결과를 평가하고 필요한 조정을 한다.
6. 성공적으로 해결된 문제에 대한 배운 점을 기록하고 다음 비슷한 문제에 대비한다.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}."
        "2. Identify the target audience, considering "
            "their interests and pain points."
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action."
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "in Korean" ,
    agent=planner,
)

Hello, how can I help you today?

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}."
        "2. Incorporate SEO keywords naturally."
        "3. Sections/Subtitles are properly named "
            "in an engaging manner."
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion."
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice."
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "in Korean",
    agent=writer,
)

I am ready to help with editing. Please provide the text that needs editing.

### Task: edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs."
                    "in Korean",
    agent=editor
)

## Creating the Crew

CrewAI 객체 생성 


In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew


**참고**: LLMs는 동일한 입력에 대해 다른 출력을 제공할 수 있으므로, 여러분이 보는 것과 다를 수 있습니다.

In [11]:
result = crew.kickoff(inputs={"topic": "최고의 하루를 보내기 위한 5가지 아침습관"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 최고의 하루를 보내기 위한 5가지 아침습관.2. Identify the target audience, considering their interests and pain points.3. Develop a detailed content outline including an introduction, key points, and a call to action.4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer:

최고의 하루를 보내기 위한 5가지 아침습관

목차:

I. 소개
- 아침습관의 중요성
- 최고의 하루를 위한 아침습관 소개

II. 5가지 아침습관
1. 일어나서 첫 번째로 하는 것
2. 건강한 아침식사
3. 몸을 움직이는 습관
4. 명상 또는 긍정적인 생각으로 시작하기
5. 계획 세우기

III. 타겟 오디언스 분석
- 바쁜 직장인
- 학생
- 집중력을 높이고 싶은 사람들

IV. SEO 키워드
- 최고의 아침습관
- 건강한 아침식사
- 명상 아침습관
- 하루를 시작하는 방법
- 아침 운동

V. 자료 및 출처
- 건강 관련 웹사이트
- 명상 앱 추천
- 운동 루틴 제안

콜 투 액션:
- 이제부터 내일 아침부터 새로운 아침습관을 실천해보세요. 최고의 하루를 보내기 위한 아침습관을 통해 더 건강하고 행복한 삶을 살아보세요. 

(End of final answer)

> Finished chain.
 [DEBUG]: == [Content Planner] Task output: 최고의 하루를 보내기 위한 5가지

In [ ]:
from IPython.display import Markdown
Markdown(result)

In [ ]:
topic = "30대에 꼭 해야할 10가지 일"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on 30대에 꼭 해야할 10가지 일.2. Identify the target audience, considering their interests and pain points.3. Develop a detailed content outline including an introduction, key points, and a call to action.4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
콘텐츠 계획서: 30대에 꼭 해야할 10가지 일

개요:
이 블로그 기사는 30대에 꼭 해야할 10가지 일에 대한 정보를 제공하고, 독자들이 무엇을 배우고 정보에 기반한 결정을 내릴 수 있도록 도와줍니다. 최신 트렌드, 핵심 인물, 주목할만한 뉴스를 중점으로 다뤄야 합니다.

대상 독자:
이 기사의 대상 독자는 30대이며, 자신의 인생과 직장에서 더 나은 방향을 찾고 있는 사람들입니다. 이들의 흥미와 고통 포인트를 고려하여 콘텐츠를 계획해야 합니다.

세부 내용:
1. 서론
- 30대에 꼭 해야할 10가지 일의 중요성 강조
- 독자들이 왜 이 기사를 읽어야 하는지 소개

2. 핵심 포인트
- 자기계발을 위한 노력 (예: 책 읽기, 운동하기)
- 금융 계획 세우기 (예: 저축, 투자)
- 건강 관리 (예: 규칙적인 검진, 운동)
- 직업 전문성 향상 (예: 교육, 자격증 취득)
- 가족 시간 확보 (예: 가족과의 교류, 휴가 계획)
- 사회 참여 (예: 자원봉사, 지역 사회 활동)
- 문화 예술 감상 (예: 공연 관람, 전시회

In [46]:
Markdown(result)

{
  "name": "City View at METREON",
  "address": "SoMa, San Francisco",
  "capacity": 2000,
  "booking_status": "Available"
}

# 2. 이벤트 계획 자동화
- workflow를 구성하여 원하는 계획을 체계적으로 구성

In [63]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기

In [65]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import load_dotenv
import os
load_dotenv()

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
gemini = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## 에이전트 만들기

In [67]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm = gpt
    
)

In [68]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=gpt
    
)

In [69]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=gemini
)

## [Venue pydantic 오브젝트 생성- pydantic 베이스모델](https://docs.pydantic.dev/latest/api/base_model/)을 사용하여 `VenueDetails` 클래스를 생성합니다.
- 상담원은 이 객체의 여러 인스턴스를 생성하여 다양한 장소에 대한 정보로 이 객체를 채웁니다.

In [70]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## 작업 만들기- output_json`을 사용하면 원하는 출력의 구조를 지정할 수 있습니다.
- 출력_파일`을 사용하면 출력을 파일로 가져올 수 있습니다
.- '휴먼_입력=참'으로 설정하면 작업이 완료되기 전에 사람의 피드백(결과 마음에 드는지 여부)을 묻습니다.

In [71]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- 비동기 실행=False`를 설정하면 작업이 그 뒤에 오는 작업과 병렬로 실행될 수 있음을 의미합니다.

In [72]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [73]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## 크루 생성하기

**참고**: 물류_작업` 및 `마케팅_작업` 작업에 대해 `async_execution=True`를 설정했으므로 이제 `작업` 목록에서 작업 순서는 중요하지 않습니다.

In [74]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2024-07-05 04:36:03,713 - 25224 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## crew 운영-  실행을 위한 입력을 설정

In [75]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**참고 1**: LLM은 동일한 입력에 대해 서로 다른 출력을 제공할 수 있으므로 동영상에 표시되는 것과 실제 결과가 다를 수 있습니다.

**참고 2**:- 일부 작업에 대해 `human_input=False`를 설정하면 실행이 완료되기 전에 실행이 사용자의 입력을 요청합니다.- 피드백을 요청하면 마우스 포인터를 사용하여 텍스트 상자를 먼저 클릭한 다음 아무 것도 입력하지 마세요.

In [76]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to start by researching venues in San Francisco that could potentially meet the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "Tech Innovation Conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: Top 16 Must-Visit San Francisco Tech Events in 2024
Link: https://startupvoyager.com/tech-events-san-francisco/
Snippet: 1. Tech Summit Silicon Valley ... Tech Summit Silicon Valley is an immersive two-day event offering a blend

This is the agent final answer: Venue - San Francisco Tech Summit
Location: Santa Clara Convention Center, Santa Clara
Date: 19th June 2024
Details: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections. The event provides networking opportunities, masterclasses, workshops, and clinics for attendees to strengthen their knowledge. Previous speakers include Pinterest's Tech Lead Thothathri Srinivasan, YouTube CEO Susan Wojcicki, and Guy Kawasaki, former Chief Evangelist at Apple. The summit is known for its Artificial Intelligence-powered matchmaking recommendations, access to over 200 premium sessions, and pre-event networking opportunities. This venue at the Santa Clara Convention Center in Santa Clara is the perfect location for the Tech Innovation Conference.
Please provide a feedback:  good


Final Answer: Venue - San Francisco Tech Summit
Location: Santa Clara Convention Center, Santa Clara
Date: 19th June 2024
Details: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections. The event provides networking opportunities, masterclasses, workshops, and clinics for attendees to strengthen their knowledge. Previous speakers include Pinterest's Tech Lead Thothathri Srinivasan, YouTube CEO Susan Wojcicki, and Guy Kawasaki, former Chief Evangelist at Apple. The summit is known for its Artificial Intelligence-powered matchmaking recommendations, access to over 200 premium sessions, and pre-event networking opportunities. This venue at the Santa Clara Convention Center in Santa Clara is the perfect location for the Tech Innovation Conference.

> Finished chain.
 [DEBUG]: == [Venue Coordinator] Task output: {
  "name": "Santa Clara Convention Center",
  "address": "Santa Clara",
  "capacity": 200,
  "

Exception in thread Thread-69 (_execute):
Traceback (most recent call last):
  File "D:\miniconda\envs\crewai\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "D:\miniconda\envs\crewai\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "D:\miniconda\envs\crewai\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "D:\miniconda\envs\crewai\lib\site-packages\crewai\task.py", line 182, in _execute
    result = agent.execute_task(
  File "D:\miniconda\envs\crewai\lib\site-packages\crewai\agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
  File "D:\miniconda\envs\crewai\lib\site-packages\langchain\chains\base.py", line 166, in invoke
    raise e
  File "D:\miniconda\envs\crewai\lib\site-packages\langchain\chains\base.py", line 156, in invoke
    self._call(inputs, run_manager=run_manager)
  File "D:\miniconda\envs\crewai\lib\site-packages\crewai\agent

I need to come up with effective strategies to promote the Tech Innovation Conference and engage potential attendees.

Action: Read website content
Action Input: {"website_url": "www.techinnovationconference.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'www.techinnovationconference.com': No scheme supplied. Perhaps you meant https://www.techinnovationconference.com?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to find effective strategies to promote the Tech Innovation Conference and engage potential attendees.

Action: Search the internet
Action Input: {"search_query": "effective strategies to promote a tech conference"} 


Search results: Title: 12 Conference Marketing Ideas to Give Your Events a Boost - Cvent
Link: https://www.cvent.com/en/blog/events/conference-marketing-ideas
Snippet: 1. Identify Your Target Audience

- 생성된 `venue_details.json` 파일을 표시합니다.

In [44]:
import json
from pprint import pprint

with open('venue_details.json') as f:
    data = json.load(f)

pprint(data)

{'address': 'SoMa, San Francisco',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- 생성된 `marketing_report.md` 파일을 표시합니다.**참고**: 킥오프` 실행이 성공적으로 실행된 후 `marketing_report.md` 파일이 생성될 때까지 45초 정도 더 기다리세요. 파일이 생성되기 전에 아래 코드를 실행하면 다음과 같은 결과가 출력됩니다:```마케팅_보고서.md```이 출력이 표시되면 조금 더 기다렸다가 다시 시도하세요.

In [45]:
from IPython.display import Markdown
Markdown("marketing_report.md")

The Tech Innovation Conference hosted by Forrester Research is the most suitable event for our company to participate in. The conference is focused on the latest advancements in technology and innovation, which aligns with our company's goals of staying at the forefront of the industry. The conference also provides opportunities for networking and collaboration with other industry leaders, which can be beneficial for our business.

To promote our participation in the conference, we will use a variety of marketing strategies, including:

* Create a dedicated landing page on our website with information about our participation in the conference, including the date, time, and location of the event, as well as a link to register for the event.
* Share information about the conference on our social media channels, using relevant hashtags and keywords to reach our target audience.
* Send out email newsletters to our subscribers with information about the conference and a link to register.
* Place ads on relevant websites and publications to promote our participation in the conference.
* Reach out to industry influencers and bloggers to see if they would be interested in writing about our participation in the conference.

We believe that these marketing strategies will help us to reach our target audience and generate interest in our participation in the Tech Innovation Conference.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os
load_dotenv()

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
gemini = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## 크루AI 도구

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## 에이전트 만들기

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool],
    llm =gemini
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool],
    llm =gemini
)

## 작업 만들기

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

- '프로세스' 클래스는 워크플로를 상담원에게 위임하는 데 도움이 됩니다(직장에서의 매니저처럼).
- 아래 예시에서는 이를 계층적으로 실행합니다.
- 'manager_llm'을 사용하면 사용하려는 "manager" LLM을 선택할 수 있습니다.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Crew 운영 -  실행을 위한 입력을 설정

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**참고**: LLM은 동일한 입력에 대해 다른 출력을 제공할 수 있으므로 동영상에 표시되는 것과 실제 결과가 다를 수 있습니다.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements for AAPL stock."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL in order to analyze trends and predict market movements effectively.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Qu

- 최종 결과를 마크다운으로 표시합니다.

In [16]:
from IPython.display import Markdown
Markdown(result)

**Comprehensive Risk Analysis Report for AAPL Trading Strategies**

**Trend Following Strategy**

* **Risks:**
    * **Whipsaw movements:** The market can quickly change direction, causing the trader to enter and exit positions multiple times, resulting in losses.
    * **False signals:** The strategy can generate false signals, leading to losses.
    * **Trend reversals:** The trend may reverse, leading to losses if the trader does not exit the position in time.
* **Mitigation strategies:**
    * **Use a stop-loss order:** To limit losses in case of a trend reversal.
    * **Use a trailing stop-loss order:** To protect profits as the trend continues.
    * **Use a moving average:** To confirm the trend before entering a position.

**Moving Average Crossover Strategy**

* **Risks:**
    * **Lagging indicator:** Moving averages are lagging indicators, which means they may not reflect the current market conditions.
    * **False signals:** The strategy can generate false signals, leading to losses.
    * **Parameter risk:** The choice of moving average periods can affect the performance of the strategy.
* **Mitigation strategies:**
    * **Use multiple moving averages:** To reduce the risk of false signals.
    * **Use a stop-loss order:** To limit losses in case of a trend reversal.
    * **Use a confirmation indicator:** To confirm the trend before entering a position.

**Support and Resistance Strategy**

* **Risks:**
    * **False breakouts:** The price may only breach a support or resistance line for a short period, leading to losses if the trader enters a position based on a false breakout.
    * **Reversals:** The price may reverse after reaching a support or resistance level, leading to losses if the trader does not exit the position in time.
    * **High volatility:** The price may experience high volatility near support and resistance levels, leading to losses if the trader is not careful.
* **Mitigation strategies:**
    * **Use a stop-loss order:** To limit losses in case of a false breakout or reversal.
    * **Use a confirmation indicator:** To confirm the support or resistance level before entering a position.
    * **Trade with the trend:** To increase the probability of a successful trade.

**Bollinger Bands Strategy**

* **Risks:**
    * **False signals:** The strategy can generate false signals, leading to losses.
    * **Parameter risk:** The choice of Bollinger Band parameters can affect the performance of the strategy.
    * **Extreme market conditions:** Bollinger Bands may not be effective in extreme market conditions, such as during a market crash or a bull market.
* **Mitigation strategies:**
    * **Use multiple Bollinger Bands:** To reduce the risk of false signals.
    * **Use a stop-loss order:** To limit losses in case of a false signal.
    * **Use a confirmation indicator:** To confirm the trend before entering a position.

**RSI Strategy**

* **Risks:**
    * **Divergence:** The RSI may diverge from the price, leading to losses if the trader enters a position based on the RSI divergence.
    * **False signals:** The strategy can generate false signals, leading to losses.
    * **Parameter risk:** The choice of RSI parameters can affect the performance of the strategy.
* **Mitigation strategies:**
    * **Use multiple RSI indicators:** To reduce the risk of false signals.
    * **Use a stop-loss order:** To limit losses in case of a false signal.
    * **Use a confirmation indicator:** To confirm the trend before entering a position.

**Insights from Data Analyst:**
Based on the search results obtained, it is evident that the strategies involving Bollinger Bands, Moving Average Crossover, Support and Resistance, and RSI are widely discussed and utilized in the trading community for AAPL. These strategies often involve combining various technical indicators to identify trends, potential reversals, and overbought or oversold conditions. However, to determine the accuracy and reliability of the data used in these strategies, a more in-depth analysis and comparison of the historical performance of these indicators with actual market movements for AAPL would be necessary. It is recommended to conduct a thorough backtesting and validation process to ensure the effectiveness of these trading strategies before making informed decisions based on them.

# 3. 사업 제휴 요청하는 메일을 보내는 Agent 만들기
> 고객을 조사하고 최근 고객의 눈높이에 맞춰서, 맞춤형으로 메일을 작성하는 에이전트를 만들어보자

In [50]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import warnings
warnings.filterwarnings('ignore')
load_dotenv()
# gemini = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_KEY"))
gpt_3 = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
gpt_4o = ChatOpenAI(model= "gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

## AGENT 계획
- sales_rep_agent : 고객에 맞는 솔루션 제공
- lead_sales_rep_agent : 대화를 리드

In [51]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="이상적인 고객 프로필과 일치하는,"
         "고부가가치 사업을 발견",
    backstory=(
        "CrewAI의 역동적인 영업팀의 일원으로서, "
        "잠재적인 사업을 디지털 환경에서 찾아내는 임무."
        "최첨단 도구와 전략적 마인드셋을 갖추고, "
        "데이터, 트렌드, 상호 작용을 분석하여 "
        "다른 사람들이 간과할 수 있는 기회를 발굴 "
        "나의 업무는 의미 있는 참여를 위한 길을 열고 "
        "회사의 성장을 이끄는 데 매우 중요함"
    ),
    llm=gpt_3,
    allow_delegation=False,
    verbose=True
)

In [52]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="개인화되고 설득력 있는 커뮤니케이션으로 사업기회 획득",
    backstory=(
        "CrewAI 영업부서의 활기찬 생태계 내에서, "
        "나는 잠재 고객과 그들이 필요로 하는 솔루션 사이의 "
        "가교 역할 수행. "
        "매력적인 메시지를 만들어 내면서, "
        "우리의 제품을 알리는 것뿐만 아니라 "
        "명확한 사업비전이 보여지고 들리는 것 같은 느낌을 준다. "
        "관심을 행동으로 전환시키는 데 있어 중추적 역할, "
        "고객의 호기심에서 헌신으로 이끌며, "
        "사업기회 제안 주도"
    ),
    llm=gpt_4o,
    allow_delegation=False,
    verbose=True
)

## Tool 만들기


- task 수준에서 도구를 전달하고 있습니다. (docs_scrape_tool)
- support_agent에 inquiry_resolution task를 전달하므로 도구를 사용할 수 있게 됩니다.
- 라이브러리, API 및 LLM 가져오기- 서퍼](https://serper.dev)

In [53]:
from crewai_tools import DirectoryReadTool,FileReadTool, SerperDevTool

In [54]:
directory_read_tool = DirectoryReadTool(directory='./instructions',)
file_read_tool = FileReadTool()
search_tool = SerperDevTool(api_key = os.getenv("SERPER_API_KEY"))

### 사용자 지정 도구
-  [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) 클래스를 사용하여 사용자 지정 도구 만들기

- 간소화 및 수업 목적을 위해 `SentimentAnalysisTool`은 모든 텍스트에 대해 `긍정`을 반환합니다.- 로컬에서 실행할 때는 `_run` 함수에서 로직으로 코드를 사용자 정의할 수 있습니다.

In [55]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = "Analyzes the sentiment of text to ensure positive and engaging communication."

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        # Perform sentiment analysis on the input text
        if "positive keyword" in text:
            return "positive"
        elif "negative keyword" in text:
            return "negative"
        else:
            return "neutral"

In [56]:
sentiment_analysis_tool = SentimentAnalysisTool()

## 작업 만들기


In [57]:
lead_profiling_task = Task(
    description=(
        "회사 {lead_name}에 대한 분석 수행 "
        "{lead_name}은 {industry} 부문의 회사로 "
        "최근 우리 솔루션에 관심을 보였음 "
        "모든 가용한 데이터 소스를 활용하여 "
        "최근 비즈니스 동향, "
        "우리 제품과 잘 맞는 잠재적 수요에 초점을 맞춘 "
        "상세한 프로필을 작성 "
        "이 작업은 우리의 참여 전략을 효과적으로 맞추는데 매우 중요."
        "가정하지 말고 "
        "확실한 정보만 사용"
    ),
    expected_output=(
        "{lead_name}에 대한 포괄적 보고서 작성 "
        "회사 배경, 주요 인력, 최근 방향성, "
        "예상되는 니즈 분야 등 포함 "
        "instruction에 나온 우리 회사의 최근 성과, 맞춤형 솔루션을 통한"
        "파트너사와 당사와의 명확한 value-propagation 강조"
        "우리 솔루션이 가치를 제공할 수 있는 "
        "잠재적 가치를 강조, "
        "customized 협업 전략 제안"
        "한국어로 작성"
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- 개인화된  작업은 사용자 지정 도구인 'SentimentAnalysisTool'과 크루AI의 'SerperDevTool'(search_tool)을 사용하고 있습니다.

In [58]:
personalized_outreach_task = Task(
    description=(
        "{lead_name} 회사에 대한 "
        "lead_profiling_task에서 얻은 통찰력 활용 "
        "{lead_name} 회사의 {position}인 {key_decision_maker}를 "
        "대상으로 한 customized outrich campaign을 생성 "
        "캠페인에는 그들의 최근 {milestone}과 "
        "우리 솔루션이 그들의 목표를 지원할 수 있는 방법을 다루어야 함 "
        "나의 커뮤니케이션은 {lead_name} 회사의 "
        "기업 문화와 가치관에 공감해야 하며, "
        "그들의 비즈니스와 수요에 대한 깊은 이해를 보여주어야 함.\n"
        "가정하지 마시고 확실한 정보만 사용"
    ),
    expected_output=(
        "{lead_name} 회사에 맞춤화된 "
        "일련의 개인화된 이메일 초안을 작성 "
        "특히 {key_decision_maker}를 대상으로 할 것."
        "instruction에 나온 우리 회사의 최근 성과, 맞춤형 솔루션 이 충분히 설명"
        "맞춤형 솔루션을 기반으로 고객에 맞는 가치제안"
        "각 초안에는 우리 솔루션과 그들의 최근 성과 및 미래 목표를 "
        "연결하는 설득력 있는 내러티브가 포함되어야 함 "
        "톤은 매력적이고 전문적이며 {lead_name} 회사의 "
        "기업 정체성과 일치해야 함"
        "한국어로 작성"
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## crew 생성 및 실행

In [59]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],

    verbose=2,
    memory=True
)

2024-07-05 05:02:19,773 - 25136 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [60]:
inputs = {
    "lead_name": "현대백화점",
    "industry":"Wholesale and retail trade",
    "key_decision_maker": "고경표",
    "position": "부장",
    "milestone": "사업제휴 협업"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: 회사 현대백화점에 대한 분석 수행 현대백화점은 Wholesale and retail trade 부문의 회사로 최근 우리 솔루션에 관심을 보였음 모든 가용한 데이터 소스를 활용하여 최근 비즈니스 동향, 우리 제품과 잘 맞는 잠재적 수요에 초점을 맞춘 상세한 프로필을 작성 이 작업은 우리의 참여 전략을 효과적으로 맞추는데 매우 중요.가정하지 말고 확실한 정보만 사용


> Entering new CrewAgentExecutor chain...
I need to gather detailed information about Hyundai Department Store, including background, key personnel, recent trends, expected needs, and potential demand areas, in order to align our participation strategy effectively with their interests.

Action: List files in directory
Action Input: {} 

File paths: 
-./instructions/Customer Engagement Guide for Small Businesses.txt
- ./instructions/Customized outrich campaign.txt
- ./instructions/Personalized Outreach Instructions for Tech Startups.txt
- ./instructions/Strategy and Innovation Framework for Enterprise Solutions.txt

Final Answer: 

현대백화점은 대한민국의 주요 백화점 체인 중 하나로, 소매 및 도매업 분야에서 활동하고 있습니다. 회사의 배경은 국내 소비자들 사이에서 높은 

2024-07-05 05:02:27,602 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-07-05 05:02:27,602 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-07-05 05:02:27,603 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')




> Entering new CrewAgentExecutor chain...
먼저, 현대백화점의 최근 사업제휴 협업과 관련된 정보를 수집하기 위해 인터넷 검색을 수행하겠습니다. 그들이 최근 어떤 비즈니스 성과를 이루었는지 파악하는 것이 중요합니다.

Action: Search the internet
Action Input: {"search_query": "현대백화점 최근 사업제휴 협업"}


2024-07-05 05:02:30,369 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-07-05 05:02:30,369 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-07-05 05:02:30,369 - 25136 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-4o to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')


 


Search results: Title: 현대백화점, 자체 개발한 캐릭터 흰디…협업으로 소비자 접점 넓혀
Link: https://www.hankyung.com/article/2024062622121
Snippet: 현대백화점, 자체 개발한 캐릭터 흰디…협업으로 소비자 접점 넓혀, 흰디와 젤리씨앗단 젤리 BGF리테일과 함께 출시 외부 유통채널에 첫 진출.
---
Title: 현대百·CU, 이번엔 캐릭터 협업 - 매일경제
Link: https://www.mk.co.kr/news/business/11038695
Snippet: 두 회사 간 협업은 지난해 말부터 시작됐다. 현대백화점그룹은 BGF리테일과 제휴를 맺어 H포인트를 전국 1만8000여 개 CU 매장에서 현금 대신 사용할 수 ...
---
Title: CU, 현대백화점과 캐릭터 IP 협업..."고객 접점 확장" - 우먼타임스
Link: http://www.womentimes.co.kr/news/articleView.html?idxno=76322
Snippet: 편의점 CU가 현대백화점과 손잡고 캐릭터 IP 협업을 통한 제휴 확장에 나선다고 11일 밝혔다.이번 제휴로 CU와 현대백화점은 전국 1만8000.
---
Title: 현대百·홈플러스 전략적 제휴 - 어패럴뉴스
Link: http://m.apparelnews.co.kr/news/news_view/?idx=48485
Snippet: 현대백화점은 최근 미래 전략사업으로 추진 중인 복합쇼핑몰의 영업 효율성을 높이기 위해 홈플러스와 전략적 제휴 ... 협력할 계획이다. 양사 제휴로 국내 유통 시장 ...
---
Title: [유통가 레이더] "협업에 경계 없다"... 현대백화점그룹, '오픈 ...
Link: https://www.greened.kr/news/articleView.html?idxno=306902
Snippet: 단순 재무적 투자가 아니라 그룹의 기존 사업과 실질적인 시너지 창출이 가능한 사업모델 발굴 차원에서 이뤄졌다. 최근에는 디지털 헬

- 최종 결과를 마크다운으로 표시합니다.

In [61]:
from IPython.display import Markdown
Markdown(result)


---

**이메일 초안 1:**

제목: 현대백화점의 성공적인 협업과 CrewAI의 맞춤형 솔루션 제안

안녕하세요, 고경표 부장님,

저는 CrewAI의 김민수입니다. 현대백화점의 최근 성과와 협업 소식을 접하고 이렇게 연락드리게 되었습니다.

특히, BGF리테일과의 협업을 통해 캐릭터 '흰디'와 젤리씨앗단 젤리 출시 및 외부 유통채널 진출 소식은 매우 인상 깊었습니다. 또한, CU와의 협업을 통해 전국 1만8000여 개 CU 매장에서 H포인트를 현금처럼 사용할 수 있게 한 전략도 훌륭합니다.

CrewAI는 이러한 현대백화점의 혁신적인 접근과 고객 중심의 전략에 깊이 공감하며, 귀사의 디지털 전환 및 고객 경험 향상을 위해 맞춤형 솔루션을 제안드리고자 합니다. 저희 AI 기반 솔루션은 고객 데이터 분석을 통해 더욱 개인화된 마케팅과 맞춤형 서비스 제공을 가능하게 합니다. 이를 통해 현대백화점의 고객 만족도를 높이고, 더 나아가 매출 증대에도 기여할 수 있을 것입니다.

고경표 부장님께서 현대백화점의 디지털 혁신을 이끌어 나가시는 데 있어, 저희 CrewAI가 신뢰할 수 있는 파트너가 될 수 있기를 기대합니다. 더 자세한 논의를 위해 미팅 일정을 잡고 싶습니다. 편하신 시간 알려주시면 감사드리겠습니다.

감사합니다.

김민수 드림  
CrewAI 영업부

---

**이메일 초안 2:**

제목: 현대백화점의 비즈니스 성과와 CrewAI의 협력 제안

안녕하세요, 고경표 부장님,

CrewAI의 이지훈입니다. 현대백화점의 최근 비즈니스 성과와 다양한 협업 프로젝트를 보며 깊은 인상을 받았습니다.

특히, 일본 파르코와의 협업을 통해 K패션 및 엔터테인먼트 브랜드의 글로벌 진출을 추진하는 모습은 귀사의 글로벌 전략을 잘 보여주는 사례라고 생각합니다. 또한, 홈플러스와의 전략적 제휴를 통해 복합쇼핑몰의 영업 효율성을 높이려는 노력도 매우 인상적이었습니다.

저희 CrewAI는 이러한 현대백화점의 혁신적인 비즈니스 전략을 지원할 수 있는 맞춤형 솔루션을 제공합니다. 저희의 AI 기반 데이터 분석 솔루션은 고객 행동 패턴을 분석하여 더 나은 의사 결정을 할 수 있도록 도와드리며, 이를 통해 고객 만족도를 극대화하고, 비즈니스 성과를 더욱 높일 수 있습니다.

고경표 부장님께 현대백화점과 CrewAI의 협력 가능성을 논의할 수 있는 기회를 주신다면 감사하겠습니다. 편하신 시간에 간단한 미팅을 요청드리오니, 가능한 시간을 알려주시면 준비하겠습니다.

감사합니다.

이지훈 드림  
CrewAI 영업부

---

**이메일 초안 3:**

제목: 현대백화점의 혁신 여정과 CrewAI의 지원 제안

안녕하세요, 고경표 부장님,

저는 CrewAI의 박지훈입니다. 현대백화점의 최근 다양한 협업과 혁신적인 비즈니스 전략을 보며 감탄했습니다.

특히, 한양대학교 창업지원단과의 산학협력을 통해 O2O(Online to Offline) 비즈니스 모델을 추진하는 모습에서 귀사의 혁신적인 사고와 전략을 엿볼 수 있었습니다. 또한, 디지털 헬스케어 스타트업과의 협업을 통해 새로운 시장을 개척하는 노력도 매우 인상적이었습니다.

저희 CrewAI는 현대백화점의 이러한 혁신적인 여정을 지원하기 위해 최적의 솔루션을 제공할 수 있습니다. AI 기반 플랫폼을 통해 고객 데이터를 분석하고, 이를 바탕으로 맞춤형 마케팅 전략을 제안하여 고객 참여와 만족도를 높이는 데 기여할 수 있습니다.

고경표 부장님과 현대백화점의 목표 달성을 위해 저희 CrewAI가 함께할 수 있는 기회를 주신다면 큰 영광이겠습니다. 편하신 시간에 미팅 일정을 조율하고자 하오니, 가능한 시간을 알려주시면 감사하겠습니다.

감사합니다.

박지훈 드림  
CrewAI 영업부

---

이상입니다. 각 이메일 초안은 현대백화점의 최근 성과와 협업을 바탕으로 맞춤형 솔루션을 제안하며, 현대백화점의 기업 문화와 가치관에 부합하는 매력적이고 전문적인 톤으로 작성되었습니다.

# 재무 분석을 통한 주식거래전략
> 특정 종목에 대해 시장을 예측하는 주식

In [62]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
import os
load_dotenv()
# gpt = ChatOpenAI(model='gpt-3.5', api_key=os.getenv("OPENAI_API_KEY")) # Loading GPT-3.5
clud = ChatAnthropic(model="claude-3-5-sonnet-20240620",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)
groq = ChatGroq(model_name="llama3-70b-8192",groq_api_key="GROQ_API_KEY",temperature=0)
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Agent 계획
- data_analyst_agent
- trading_strategy_agent
- risk_management_agent

In [63]:
from crewai_tools import tool
from crewai_tools import WebsiteSearchTool, SerperDevTool,ScrapeWebsiteTool,DirectoryReadTool,FileReadTool

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrape_tool = ScrapeWebsiteTool()
docs_tool = DirectoryReadTool(directory='./financial-posts')
file_tool = FileReadTool()

class DataTools():
    @tool("get historical stock price")
    def get_stock_price(ticker: str, start_date: str, end_date: str) -> str:
        """
        Get the stock price for a given ticker symbol and date range.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)
            start_date (str): The start date in the format 'YYYY-MM-DD'
            end_date (str): The end date in the format 'YYYY-MM-DD'

        Returns:
            str: The stock price data for the given ticker and date range.
        """        
        import yfinance as yf
        import datetime

        today = datetime.datetime.now()
        one_year_ago = today - datetime.timedelta(days=365)

        stock_data = yf.download(ticker, start=one_year_ago, end=today)
        stock_price_data = stock_data['Adj Close'].to_string()

        return stock_price_data

    @tool("get financial statement")        
    def get_financial_statement(ticker: str) -> str:
        '''
        Get the quarterly financial statement for a given ticker symbol.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)

        Returns:
            str: quarterly_financials
        '''
        import pandas as pd
        import yfinance as yf
        ticker = yf.Ticker(ticker)
        return ticker.quarterly_financials

# def cache_func(args, result):
#     # Cache the result if the ticker is a valid stock symbol
#     ticker, start_date, end_date = args
#     try:
#         yf.download(ticker, start=start_date, end=end_date)
#         cache = True
#     except:
#         cache = False
#     return cache

# DataTools.get_stock_price.cache_function = cache_func
# DataTools.get_financial_statement.cache_function = cache_func


In [64]:
data_analyst_agent = Agent(
    role="data_analyst_agent",
    goal="실시간으로 시장 데이터를 모니터링하고 분석하여 "
         "트렌드를 식별하고 시장 움직임을 예측",
    backstory="금융 시장 전문가로서 "
              "통계 모델링과 기계 학습을 사용하여 "
              "중요한 통찰력을 제공합니다. 데이터 분석 능력으로"
              "거래 결정을 알리는 데"
              "핵심적인 역할을 합니다.",
    tools=[search_tool,DataTools.get_stock_price,DataTools.get_financial_statement],
    verbose=True,
    allow_delegation=True,
    llm=clud
)

In [65]:
trading_strategy_agent = Agent(
    role="trading_strategy_agent",
    goal="data_analyst_agent의 분석 기반, "
         "다양한 트레이딩 전략을 개발하고 테스트",
    backstory="금융 시장과 정량 분석에 대한 깊은 이해를 갖추고 "
              "트레이딩 전략을 고안하고 개선합니다. "
              "다양한 접근 방식의 성과를 평가하여 "
              "가장 수익성이 높고 위험 회피적인 옵션을 결정합니다.",
    tools=[scrape_tool],
    verbose=True,
    allow_delegation=True,
    llm=groq
)

In [66]:
risk_management_agent = Agent(
    role="risk_management_agent",
    goal="투자 전략에 있어서 잠재적 트레이딩 활동과 관련된 리스크를 "
         "평가하고 통찰력을 제공합니다.",
    backstory="리스크 평가 모델과 시장 동향에 대한 깊은 이해를 바탕으로 "
              "제안된 트레이드의 잠재적 리스크를 꼼꼼히 살펴봅니다. "
              "리스크 노출에 대한 상세한 분석을 제공하고 "
              "트레이딩 활동이 회사의 리스크 허용 범위 내에서 이루어지도록 "
              "보호 장치를 제안합니다.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=True,
    llm=clud
)

## 작업 만들기

In [67]:
# 데이터 분석가 에이전트를 위한 작업: 시장 데이터 분석
data_analysis_task = Task(
    description=(
        "선택된 주식({stock_selection})에 대한 "
        "시장 데이터를 지속적으로 모니터링하고 분석."
        "통계 모델링과 기계 학습을 사용하여 "
        "트렌드를 식별하고 시장 움직임을 예측."
    ),
    expected_output=(
        "{stock_selection}에 대한 "
        "중요한 시장 기회 또는 위협에 관한 인사이트 및 경고"
    ),
    agent=data_analyst_agent,
)

In [68]:
# 트레이딩 전략 에이전트를 위한 작업: 트레이딩 전략 개발
strategy_development_task = Task(
    description=(
        "trading_strategy_agent의 통찰력과 "
        "사용자 정의 리스크 허용 범위({risk_tolerance})를 바탕으로 "
        "트레이딩 전략을 개발하고 개선하세요. "
        "트레이딩 선호도({trading_strategy_preference})를 고려하세요."
    ),
    expected_output=(
        "{stock_selection}에 대한 "
        "사용자의 리스크 허용 범위에 부합하는 "
        "잠재적 트레이딩 전략 세트입니다."
    ),
    agent=trading_strategy_agent,
)

In [69]:
# 리스크 어드바이저 에이전트를 위한 작업: 트레이딩 리스크 평가
risk_assessment_task = Task(
    description=(
        "{stock_selection}에 대한 제안된 트레이딩 전략과 "
        "실행 계획과 관련된 리스크를 평가하세요. "
        "잠재적 리스크에 대한 상세한 분석을 제공하고 "
        "리스크 완화 전략을 제안하세요."
    ),
    expected_output=(
        "{stock_selection}에 대한 잠재적 리스크와 "
        "리스크 완화 권고 사항을 상세히 설명한 "
        "포괄적인 리스크 분석 보고서입니다."
    ),
    agent=risk_management_agent,
)

## Crew 생성
- 'Process' 클래스는 워크플로를 상담원에게 위임하는 데 도움이 됩니다(직장에서의 매니저처럼).
- 아래 예시에서는 이를 계층적으로 실행합니다.
- 'manager_llm'을 사용하면 사용하려는 "manager" LLM을 선택할 수 있습니다.

In [70]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           risk_assessment_task],
    
    manager_llm=clud,

    process=Process.hierarchical,
    verbose=True
)

2024-07-05 05:03:33,936 - 25136 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Crew  실행

In [71]:
# 프로세스를 시작해봅시다
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '2500',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Monthly Trading',
    'news_impact_consideration': True
}

In [72]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 선택된 주식(TSLA)에 대한 시장 데이터를 지속적으로 모니터링하고 분석.통계 모델링과 기계 학습을 사용하여 트렌드를 식별하고 시장 움직임을 예측.


> Entering new CrewAgentExecutor chain...
Thought: To complete this task effectively, I need to delegate work to our data analyst and trading strategy agent. I'll start by asking the data analyst to gather and analyze market data for TSLA, and then have the trading strategy agent use that information to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "data_analyst_agent", "task": "Gather and analyze market data for TSLA", "context": "We need to continuously monitor and analyze market data for Tesla (TSLA) stock. Please collect relevant data such as stock price, trading volume, market cap, P/E ratio, and any other significant financial metrics. Also, gather news and sentiment data related to TSLA. Analyze this data to identify any notable patterns or anomalies."}


> Entering new Cr

[*********************100%%**********************]  1 of 1 completed


 

Date
2023-07-06    276.540009
2023-07-07    274.429993
2023-07-10    269.609985
2023-07-11    269.790009
2023-07-12    271.989990
2023-07-13    277.899994
2023-07-14    281.380005
2023-07-17    290.380005
2023-07-18    293.339996
2023-07-19    291.260010
2023-07-20    262.899994
2023-07-21    260.019989
2023-07-24    269.059998
2023-07-25    265.279999
2023-07-26    264.350006
2023-07-27    255.710007
2023-07-28    266.440002
2023-07-31    267.429993
2023-08-01    261.070007
2023-08-02    254.110001
2023-08-03    259.320007
2023-08-04    253.860001
2023-08-07    251.449997
2023-08-08    249.699997
2023-08-09    242.190002
2023-08-10    245.339996
2023-08-11    242.649994
2023-08-14    239.759995
2023-08-15    232.960007
2023-08-16    225.600006
2023-08-17    219.220001
2023-08-18    215.490005
2023-08-21    231.279999
2023-08-22    233.190002
2023-08-23    236.860001
2023-08-24    230.039993
2023-08-25    238.589996
2023-08-28    238.820007
2023-08-29    257.179993
2023-08-30    256

- 최종 결과를 마크다운으로 표시합니다.

In [73]:
from IPython.display import Markdown
Markdown(result)

TSLA Trading Strategy Risk Analysis Report

This comprehensive risk analysis report outlines the potential risks associated with the proposed trading strategies for TSLA and provides detailed risk mitigation recommendations.

1. Market Volatility and Price Fluctuation Risks

Potential Risks:
- TSLA stock has demonstrated significant volatility, with a 102% gain in 2023 followed by a 30.4% drop in early 2024.
- This volatility poses risks for all proposed strategies, particularly the Monthly Rebalancing and Trend-Following strategies.

Risk Mitigation Recommendations:
- Implement strict stop-loss orders to limit downside risk.
- Use a longer timeframe for trend analysis in the Trend-Following Strategy to avoid false signals from short-term fluctuations.
- For the Monthly Rebalancing Strategy, consider using a volatility-adjusted rebalancing approach.

2. Valuation Concerns

Potential Risks:
- Tesla's high valuation multiples make it susceptible to significant corrections if growth expectations are not met.
- This particularly affects the Value-Based Entry Strategy.

Risk Mitigation Recommendations:
- Incorporate multiple valuation metrics beyond traditional P/E ratios, such as EV/EBITDA and PEG ratio.
- Set conservative entry points with a significant margin of safety.
- Diversify the portfolio to include other EV and tech stocks to spread risk.

3. Competition and Market Share Risks

Potential Risks:
- Increasing competition in the EV market and potential market share loss could impact Tesla's performance.
- This affects all strategies, particularly the Earnings-Based and News-Driven Momentum strategies.

Risk Mitigation Recommendations:
- Closely monitor competitive landscape and Tesla's market share trends.
- For the Earnings-Based Strategy, focus on both absolute and relative performance metrics.
- In the News-Driven Momentum Strategy, incorporate sentiment analysis tools to gauge market reaction to competitive news.

4. Production and Supply Chain Risks

Potential Risks:
- Tesla faces risks related to production capacity, battery supply, and global supply chain disruptions.
- These factors could affect its ability to meet demand and impact stock performance.

Risk Mitigation Recommendations:
- For the Margin Improvement Strategy, track Tesla's progress in optimizing production and reducing costs.
- Implement a scaling approach in position sizing based on production milestone achievements.
- Consider pairing TSLA trades with suppliers or competitors as a hedge.

5. Regulatory and Policy Risks

Potential Risks:
- Changes in government policies, subsidies, and regulations in key markets like China and the US can significantly impact Tesla's business model and stock performance.

Risk Mitigation Recommendations:
- Incorporate a geopolitical risk assessment in the decision-making process for all strategies.
- For the Options Collar Strategy, adjust the strike prices and expiration dates based on major policy announcement schedules.
- Maintain a portion of the portfolio in cash or less volatile assets to capitalize on opportunities arising from regulatory-induced dips.

6. Technological and Innovation Risks

Potential Risks:
- Tesla's stock price is heavily influenced by its perceived leadership in EV technology and innovation.
- Any setbacks or leapfrogging by competitors could lead to significant stock price movements.

Risk Mitigation Recommendations:
- For the News-Driven Momentum Strategy, develop a scoring system for technological announcements and their potential impact.
- In the Trend-Following Strategy, incorporate technical indicators that capture innovation-related momentum.
- Regularly reassess Tesla's competitive advantage in key technologies like batteries and autonomous driving.

7. Macroeconomic Risks

Potential Risks:
- Factors such as interest rates, inflation, and overall economic health can impact Tesla's stock performance and the effectiveness of the proposed strategies.

Risk Mitigation Recommendations:
- Adjust position sizes based on macroeconomic indicators.
- For the Value-Based Entry Strategy, incorporate macroeconomic factors into the valuation models.
- Consider using ETFs or index futures as a hedge against broader market movements.

8. Execution and Liquidity Risks

Potential Risks:
- While TSLA is generally a liquid stock, extreme market conditions or news events can lead to widened bid-ask spreads and execution challenges.

Risk Mit

# 주식거래봇 2

````
pip install tools sec-api unstructured
```

In [74]:
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic
from langchain.schema import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

In [75]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html
from langchain.tools import tool
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from sec_api import QueryApi

class DataTools():
    @tool("get historical stock price")
    def get_stock_price(ticker: str, start_date: str, end_date: str) -> str:
        """
        Get the stock price for a given ticker symbol and date range.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)
            start_date (str): The start date in the format 'YYYY-MM-DD'
            end_date (str): The end date in the format 'YYYY-MM-DD'

        Returns:
            str: The stock price data for the given ticker and date range.
        """        
        import yfinance as yf
        import datetime

        today = datetime.datetime.now()
        one_year_ago = today - datetime.timedelta(days=365)

        stock_data = yf.download(ticker, start=one_year_ago, end=today)
        market_data = yf.download('^GSPC', start=one_year_ago, end=today)
        monthly_data = stock_data.resample('M').agg({
            'Close': 'last',  # 월말 종가
            'Volume': 'sum'   # 월간 누적 거래량
        })
        monthly_market = market_returns.resample('M').mean()  
        monthly_data['3M_Close'] = monthly_data['Close'].rolling(window=3).mean().shift(-2)        # 3개월 이동평균 종가 계
        risk_free_rate = 0.02 / 12  # 연 2% 무위험 수익률을 월별로 환산 # 샤프지수 계산
        monthly_data['Sharpe_Ratio'] = (monthly_data['Close'].pct_change() - risk_free_rate) / monthly_data['Close'].pct_change().std()
        monthly_data['Beta'] = stock_data['Close'].pct_change().resample('M').mean()/market_data['Close'].pct_change().resample('M').mean()
        return monthly_data

    @tool("get financial statement")        
    def get_financial_statement(ticker: str) -> str:
        '''
        Get the quarterly financial statement for a given ticker symbol.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)

        Returns:
            str: quarterly_financials
        '''
        import pandas as pd
        import yfinance as yf
        ticker = yf.Ticker(ticker)
        return ticker.quarterly_financials.iloc[:,[0,4]]

class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations, 
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical 
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        return eval(operation)


class SearchTools():
    @tool("Search the internet")
    def search_internet(query):
        """Useful to search the internet 
        about a a given topic and return relevant results"""
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()['organic']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                f"Title: {result['title']}", f"Link: {result['link']}",
                f"Snippet: {result['snippet']}", "\n-----------------"
            ]))
            except KeyError:
                next

        return '\n'.join(string)
    
    @tool("Search news on the internet")
    def search_news(query):
        """Useful to search news about a company, stock or any other
        topic and return relevant results"""""
        top_result_to_return = 4
        url = "https://google.serper.dev/news"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()['news']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                    f"Title: {result['title']}", f"Link: {result['link']}",
                    f"Snippet: {result['snippet']}", "\n-----------------"
                ]))
            except KeyError:
                next

        return '\n'.join(string)



In [76]:
class StockAnalysisTasks():
    def research(self, agent, company):
        return Task(
        description=dedent(f"""
        수집하고 요약하세요. 최근 뉴스 기사, 보도 자료 및 주식과
        해당 산업과 관련된 시장 분석 자료.
        중요한 이벤트, 시장 분위기 및 애널리스트 의견에
        특별한 주의를 기울이세요. 또한 실적 발표 등의 예정된
        이벤트도 포함하세요.

        최종 답변에는 반드시 최신 뉴스에 대한 포괄적인 요약,
        주목할 만한 시장 분위기 변화, 주가에 미칠 수 있는 잠재적 영향이
        포함되어야 합니다.
        또한 주식 티커도 반환해야 합니다.

        {self.__tip_section()}

        가능한 한 최신 데이터를 사용하세요.

        고객이 선택한 회사: {company}
        """),
        expected_output= "주식과 해당 산업과 관련된 최근 뉴스, 시장 분석, 이벤트를 요약한 포괄적인 보고서. 주목할 만한 시장 분위기 변화, \
        주가에 미칠 수 있는 잠재적 영향 및 주식 티커가 포함되어야 합니다.",
        agent=agent
    )
    
    def financial_analysis(self, agent):
        expected_output = "주요 재무 지표, 업계 동종 기업 및 시장 동향과의 비교, 강점, 약점, 전반적인 재무 건전성 등을 포함하여 \
        주식의 재무 건전성과 시장 성과를 분석한 포괄적인 보고서입니다."
        return Task(
            description=dedent(f"""
            주식의 재무 건전성과 시장 성과를 철저히 분석하세요.
            이에는 PER, EPS 성장률, 매출 추이, 부채비율 등
            주요 재무 지표 검토가 포함됩니다.
            또한 업계 동종 기업 및 전반적인 시장 동향과
            비교하여 주식 성과를 분석하세요.

            최종 보고서에는 제공된 요약을 확장하되,
            현재 시장 상황에서 주식의 재무 건전성, 강점과 약점,
            경쟁사 대비 위치 등에 대한 명확한 평가가 포함되어야 합니다.
            {self.__tip_section()}

            가능한 한 최신 데이터를 사용하세요.
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def filings_analysis(self, agent):
        expected_output = "경영진의 분석, 재무제표, 내부자 거래 활동, 공시 위험 등 최신 10-Q 및 10-K 보고서의 주요 부분에서 \
        중요한 발견 사항을 강조한 확장 보고서입니다. 고객을 위한 위험 요인 또는 긍정적 지표도 포함되어 있습니다."
        return Task(
            description=dedent(f"""
            EDGAR에서 해당 주식의 최신 10-Q 및 10-K 보고서를
            분석하세요.
            경영진의 분석, 재무제표, 내부자 거래 활동,
            공시된 위험 등 주요 부분에 중점을 두세요.
            주식 미래 성과에 영향을 줄 수 있는 관련 데이터와
            통찰력을 추출하세요.

            최종 답변에는 이러한 보고서에서 중요한 발견 사항을
            강조한 확장 보고서가 포함되어야 합니다.
            고객을 위한 위험 요인 또는 긍정적 지표도 포함하세요.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def recommend(self, agent):
        expected_output = "재무 애널리스트와 리서치 애널리스트의 분석을 종합하여 재무 건전성, 시장 분위기, \
        EDGAR 보고서의 정성적 데이터 등 모든 측면을 고려한 포괄적인 투자 권장 보고서입니다. \
        보고서에는 명확한 투자 입장과 전략, 지원 근거, 내부자 거래 활동 섹션, 예정된 실적 발표 등이 포함되어야 하며 고객을 위해 잘 구성되어야 합니다."
        return Task(
            description=dedent(f"""
            재무 애널리스트와 리서치 애널리스트가 제공한 분석을
            검토하고 종합하세요.
            이러한 통찰력을 결합하여 포괄적인 투자 권장 사항을
            작성하세요.

            재무 건전성, 시장 분위기, EDGAR 보고서의 정성적
            데이터 등 모든 측면을 고려해야 합니다.

            내부자 거래 활동 섹션과 예정된 실적 발표 등도
            포함해야 합니다.

            최종 답변에는 고객을 위한 권장 사항이 포함되어야 합니다.
            지원 근거와 함께 명확한 투자 입장과 전략을 제시하는
            매우 상세한 전체 보고서여야 합니다.
            고객을 위해 예쁘고 잘 구성된 형식이어야 합니다.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def __tip_section(self):
        return "최선을 다해 작업하면 10,000달러의 수수료를 드리겠습니다!"

In [77]:
from crewai import Task
from textwrap import dedent


class StockAnalysisTasks():
    def research(self, agent, company):
        return Task(
        description=dedent(f"""
        주식 및 해당 산업과 관련된 최근 뉴스 기사, 보도자료, 시장 분석 자료를 수집하고 요약합니다.
        중요한 이벤트, 시장 분위기, 애널리스트 의견에 특별한 주의를 기울입니다.
        실적 발표 등 예정된 이벤트도 포함합니다.
        최종 답변은 다음을 포함하는 보고서여야 합니다:
            최신 뉴스에 대한 포괄적인 요약,
            주목할 만한 시장 분위기 변화,
            주가에 미칠 수 있는 잠재적 영향
        주식 티커도 반환해야 합니다.

        {self.__tip_section()}

        가능한 한 최신 데이터를 사용해야 합니다.
        고객이 선택한 company를 지정해야 합니다.: {company}
        """),
        expected_output= "A comprehensive report summarizing recent news, market analyses,\
        and events related to the stock and its industry, including notable shifts in market sentiment, \
        potential impacts on the stock, and the stock ticker.",
        agent=agent
    )
    
    def financial_analysis(self, agent):
        expected_output = "A comprehensive report analyzing the stock's financial health and market performance, \
        including key financial metrics, comparison with industry peers and \
        market trends, strengths, weaknesses, and overall financial standing."
        return Task(
            description=dedent(f"""
            주식의 재무 건전성과 시장 성과를 철저히 분석합니다.
            PER, EPS 성장률, 매출 추이, 부채비율 등 주요 재무지표를 검토합니다.
            업계 동종기업 및 전반적인 시장 동향과 비교하여 주식 성과를 분석합니다.
            최종 보고서에는 제공된 요약을 확장하되, 다음 사항을 명확히 평가해야 합니다:
                주식의 재무 상황,
                강점과 약점,
                현재 시장에서 경쟁사 대비 위치.{self.__tip_section()}
            가능한 한 최신 데이터를 사용해야 합니다.
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def filings_analysis(self, agent):
        expected_output = "An expanded report highlighting significant findings from the latest 10-Q and 10-K filings, \
        including key insights from sections like Management's Discussion and Analysis, financial statements, \
        insider trading activity, and disclosed risks, as well as any red flags or positive indicators for the customer."
        return Task(
            description=dedent(f"""
            최근 주가의 흐름을 분석합니다.
            경영진 분석, 재무제표, 내부자 거래 활동, 공시된 위험 등 핵심 섹션에 중점을 둡니다.
            주식 미래 성과에 영향을 줄 수 있는 관련 데이터와 통찰력을 추출합니다.
            최종 답변은 이러한 보고서에서 중요한 발견 사항을 강조한 확장 보고서여야 합니다.
            고객을 위한 위험 요인(레드 플래그) 또는 긍정적 지표를 포함해야 합니다.{self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def recommend(self, agent):
        expected_output = "A comprehensive investment recommendation report \
        that synthesizes the analyses from the Financial Analyst and Research Analyst, \
        considering all aspects such as financial health, market sentiment, and qualitative data from EDGAR filings. \
        The report should include a clear investment stance and strategy with supporting evidence, a section on insider trading activity, \
        upcoming events like earnings, and be well-formatted for the customer."
        return Task(
            description=dedent(f"""
            재무 애널리스트와 리서치 애널리스트가 제공한 분석을 검토하고 종합합니다.
            이러한 통찰력을 결합하여 포괄적인 투자 권장 사항을 작성합니다.
            재무 건전성, 시장 분위기, 실적 보고서의 정성적 데이터 등 모든 측면을 고려해야 합니다.
            내부자 거래 활동 섹션과 예정된 실적 발표 등을 포함해야 합니다.
            최종 답변은 고객을 위한 권장 사항이어야 합니다.
            매우 상세한 전체 보고서 형식이어야 하며, 지원 근거와 함께 명확한 투자 입장과 전략을 제시해야 합니다.
            고객을 위해 예쁘고 잘 구성된 형식이어야 합니다.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def __tip_section(self):
        return "If you do your BEST WORK, I'll give you a $10,000 commission!"

In [80]:
from crewai import Agent
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
load_dotenv()

load_dotenv()
chat = ChatGroq(model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"), temperature=0)
clud = ChatAnthropic(model="claude-3-sonnet-20240229",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)


class StockAnalysisAgents():
    def local_expert(self):
        return Agent(
            role='The Best Financial Analyst',
            goal="""Impress all customers with your financial data and market trends analysis""",
            backstory="""The most seasoned financial analyst with lots of expertise in stock market analysis and investment strategies that is working for a super important customer.""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                CalculatorTools.calculate,
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )

    def research_analyst(self):
        return Agent(
            role='Staff Research Analyst',
            goal="""Being the best at gather, interpret data and amaze your customer with it""",
            backstory="""Known as the BEST research analyst, you're skilled in sifting through news, company announcements, and market sentiments. Now you're working on a super important customer""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                YahooFinanceNewsTool(),
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )

    def investment_advisor(self):
        return Agent(
            role='Private Investment Advisor',
            goal="""Impress your customers with full analyses over stocks and completer investment recommendations""",
            backstory="""You're the most experienced investment advisor and you combine various analytical insights to formulate strategic investment advice. You are now working for a super important customer you need to impress.""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                CalculatorTools.calculate,
                YahooFinanceNewsTool()
            ]
        )

class FinancialCrew:
    def __init__(self, company):
        self.company = company

    def run(self):
        agents = StockAnalysisAgents()
        tasks = StockAnalysisTasks()

        research_analyst_agent = agents.research_analyst()
        financial_analyst_agent = agents.local_expert()
        investment_advisor_agent = agents.investment_advisor()

        research_task = tasks.research(research_analyst_agent, self.company)
        financial_task = tasks.financial_analysis(financial_analyst_agent)
        filings_task = tasks.filings_analysis(financial_analyst_agent)
        recommend_task = tasks.recommend(investment_advisor_agent)

        crew = Crew(
            agents=[
                research_analyst_agent,
                financial_analyst_agent,
                investment_advisor_agent
            ],
            tasks=[
                research_task,
                financial_task,
                filings_task,
                recommend_task
            ],
            manager_llm=chat,  # <----- Using ChatGroq as the manager
            verbose=True
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to Financial Analysis Crew")
    print('-------------------------------')
    company = input(
        dedent("""
      What is the company you want to analyze?
    """))
    financial_crew = FinancialCrew(company)
    result = financial_crew.run()
    print("\n\n########################")
    print("## Here is the Report")
    print("########################\n")
    print(result)

## Welcome to Financial Analysis Crew
-------------------------------



What is the company you want to analyze?
 TSLA


2024-07-05 05:08:15,231 - 25136 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Staff Research Analyst
 [INFO]: == Starting Task: 
주식 및 해당 산업과 관련된 최근 뉴스 기사, 보도자료, 시장 분석 자료를 수집하고 요약합니다.
중요한 이벤트, 시장 분위기, 애널리스트 의견에 특별한 주의를 기울입니다.
실적 발표 등 예정된 이벤트도 포함합니다.
최종 답변은 다음을 포함하는 보고서여야 합니다:
    최신 뉴스에 대한 포괄적인 요약,
    주목할 만한 시장 분위기 변화,
    주가에 미칠 수 있는 잠재적 영향
주식 티커도 반환해야 합니다.

If you do your BEST WORK, I'll give you a $10,000 commission!

가능한 한 최신 데이터를 사용해야 합니다.
고객이 선택한 company를 지정해야 합니다.: TSLA



> Entering new CrewAgentExecutor chain...
The task at hand is to gather and summarize the recent news, market analyses, and events related to TSLA (Tesla) and its industry. This includes notable shifts in market sentiment, potential impacts on the stock, and the stock ticker. I need to return the actual complete content as the final answer, not a summary. I will make sure to use the most recent data available.

Action: Search news on the internet
Action Input: {"query": "TSLA"} 

Title: Tesla (NASDAQ:TSLA) Braces for First Consecutive Quarterly Decline Amid R

KeyboardInterrupt: 

In [ ]:
# from crewai import Agent
# from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
# from langchain.chat_models import ChatOpenAI
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq
# import os
# load_dotenv()
# # gpt = ChatOpenAI(model='gpt-3.5', api_key=os.getenv("OPENAI_API_KEY")) # Loading GPT-3.5
# # clud = ChatAnthropic(model="claude-3-sonnet-20240229",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)
# chat = ChatGroq(model_name="mixtral-8x7b-32768",groq_api_key="GROQ_API_KEY",temperature=0)
# class StockAnalysisAgents():

#     def local_expert(self):
#         return Agent(
#                   role='The Best Financial Analyst',
#                   goal="""Impress all customers with your financial data 
#                   and market trends analysis""",
#                   backstory="""The most seasoned financial analyst with 
#                   lots of expertise in stock market analysis and investment
#                   strategies that is working for a super important customer.""",
#                   verbose=True,
#                   llm=chat, # <----- passing our llm reference here
#                   tools=[
#                     SearchTools.search_internet,
#                     CalculatorTools.calculate,
#                     # SECTools.search_10q,
#                     # SECTools.search_10k
#                     DataTools.get_stock_price,
#                     DataTools.get_financial_statement
#                   ]
#                 )
#     def research_analyst(self):
#         return Agent(
#                   role='Staff Research Analyst',
#                   goal="""Being the best at gather, interpret data and amaze
#                   your customer with it""",
#                   backstory="""Known as the BEST research analyst, you're
#                   skilled in sifting through news, company announcements, 
#                   and market sentiments. Now you're working on a super 
#                   important customer""",
#                   verbose=True,
#                   LLM=chat,
#                   tools=[
#                     SearchTools.search_internet,
#                     SearchTools.search_news,
#                     YahooFinanceNewsTool(),
#                     DataTools.get_stock_price,
#                     DataTools.get_financial_statement
#                   ]
#               )
#     def investment_advisor(self):
#         return Agent(
#                   role='Private Investment Advisor',
#                   goal="""Impress your customers with full analyses over stocks
#                   and completer investment recommendations""",
#                   backstory="""You're the most experienced investment advisor
#                   and you combine various analytical insights to formulate
#                   strategic investment advice. You are now working for
#                   a super important customer you need to impress.""",
#                   verbose=True,
#                   LLM=chat,
#                   tools=[
#                     SearchTools.search_internet,
#                     SearchTools.search_news,
#                     CalculatorTools.calculate,
#                     YahooFinanceNewsTool()
#                   ]
#                 )
    
# from crewai import Crew
# from textwrap import dedent
# from dotenv import load_dotenv
# load_dotenv()

# # from stock_analysis_agents import StockAnalysisAgents
# # from stock_analysis_tasks import StockAnalysisTasks


# class FinancialCrew:
#     def __init__(self, company):
#         self.company = company
#     def run(self):
#         agents = StockAnalysisAgents()
#         tasks = StockAnalysisTasks()

#         research_analyst_agent = agents.research_analyst()
#         financial_analyst_agent = agents.local_expert()
#         investment_advisor_agent = agents.investment_advisor()

#         research_task = tasks.research(research_analyst_agent, self.company)
#         financial_task = tasks.financial_analysis(financial_analyst_agent)
#         filings_task = tasks.filings_analysis(financial_analyst_agent)
#         recommend_task = tasks.recommend(investment_advisor_agent)

#         crew = Crew(
#           agents=[
#             research_analyst_agent,
#             financial_analyst_agent,
#             investment_advisor_agent
#           ],
#           tasks=[
#             research_task,
#             financial_task,
#             filings_task,
#             recommend_task
#           ],
#           manager_llm=clud,
#           verbose=True
#         )

#         result = crew.kickoff()
#         return result
    
# if __name__ == "__main__":
#     print("## Welcome to Financial Analysis Crew")
#     print('-------------------------------')
#     company = input(
#         dedent("""
#       What is the company you want to analyze?
#     """))
#     financial_crew = FinancialCrew(company)
#     result = financial_crew.run()
#     print("\n\n########################")
#     print("## Here is the Report")
#     print("########################\n")
#     print(result)

In [ ]:
# class SECTools():
#     def __download_form_html(url):
#         headers = {
#       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#       'Accept-Encoding': 'gzip, deflate, br',
#       'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
#       'Cache-Control': 'max-age=0',
#       'Dnt': '1',
#       'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120"',
#       'Sec-Ch-Ua-Mobile': '?0',
#       'Sec-Ch-Ua-Platform': '"macOS"',
#       'Sec-Fetch-Dest': 'document',
#       'Sec-Fetch-Mode': 'navigate',
#       'Sec-Fetch-Site': 'none',
#       'Sec-Fetch-User': '?1',
#       'Upgrade-Insecure-Requests': '1',
#       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
#     }
#         response = requests.get(url, headers=headers)
#         return response.text
    
#     def __embedding_search(url, ask):
#         text = SECTools.__download_form_html(url)
#         elements = partition_html(text=text)
#         content = "\n".join([str(el) for el in elements])
#         text_splitter = CharacterTextSplitter(
#             separator = "\n",
#             chunk_size = 1000,
#             chunk_overlap  = 150,
#             length_function = len,
#             is_separator_regex = False,
#         )
#         docs = text_splitter.create_documents([content])
#         retriever = FAISS.from_documents(
#           docs, OpenAIEmbeddings()
#         ).as_retriever()
#         answers = retriever.get_relevant_documents(ask, top_k=4)
#         answers = "\n\n".join([a.page_content for a in answers])
#         return answers
    
#     @tool("Search 10-Q")
#     def search_10q(query):
#         """
#         Useful to search information from the latest 10-Q form for a given stock.
#         The input to this tool should be a pipe (|) separated text of length two, 
#         representing the stock ticker you are interested and what question you have from it.
#         For example, 'AAPL|what was last quarter's revenue'.
#         """
#         stock, ask = query.split("|")
#         queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])
#         query = {
#           "query": {
#             "query_string": {
#               "query": f"ticker:{stock} AND formType:\"10-Q\""
#             }
#           },
#           "from": "0",
#           "size": "1",
#           "sort": [{ "filedAt": { "order": "desc" }}]
#         }

#         fillings = queryApi.get_filings(query)['filings']
#         if len(fillings) == 0:
#             return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#         link = fillings[0]['linkToFilingDetails']
#         answer = SECTools.__embedding_search(link, ask)
#         return answer

#     @tool("Search 10-K")
#     def search_10k(query):
#         """
#         Useful to search information from the latest 10-K form for a given stock.
#         The input to this tool should be a pipe (|) separated text of length two, 
#         representing the stock ticker you are interested, what question you have from it.
#         For example, 'AAPL|what was last year's revenue'.
#         """
#         stock, ask = query.split("|")
#         queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])
#         query = {
#           "query": {
#             "query_string": {
#               "query": f"ticker:{stock} AND formType:\"10-K\""
#             }
#           },
#           "from": "0",
#           "size": "1",
#           "sort": [{ "filedAt": { "order": "desc" }}]
#         }

#         fillings = queryApi.get_filings(query)['filings']
#         if len(fillings) == 0:
#             return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#         link = fillings[0]['linkToFilingDetails']
#         answer = SECTools.__embedding_search(link, ask)
#         return answer   
